## Instructions
0. If you haven't already, follow [the setup instructions here](https://jennselby.github.io/MachineLearningCourseNotes/#setting-up-python3) to get all necessary software installed.
0. Install the Gensim word2vec Python implementation: `python3 -m pip install --upgrade gensim`
0. Get the trained model (1billion_word_vectors.zip) from Canvas and put it in the same folder as this ipynb file.
0. Unzip the trained model file. You should now have three files in the folder (if zip created a new folder, move these files out of that separate folder into the same folder as this ipynb file):
    * 1billion_word_vectors
    * 1billion_word_vectors.syn1neg.npy
    * 1billion_word_vectors.wv.syn0.npy
0. Read through the code in the following sections:
    * [Load trained word vectors](#Load-Trained-Word-Vectors)
    * [Explore word vectors](#Explore-Word-Vectors)
0. Optionally, complete [Exercise: Explore Word Vectors](#Exercise:-Explore-Word-Vectors)
0. Read through the code in the following sections:
    * [Use Word Vectors in an Embedding Layer of a Keras Model](#Use-Word-Vectors-in-an-Embedding-Layer-of-a-Keras-Model)
    * [IMDB Dataset](#IMDB-Dataset)
    * [Train IMDB Word Vectors](#Train-IMDB-Word-Vectors)
    * [Process Dataset](#Process-Dataset)
    * [Classification With Word Vectors Trained With Model](#Classification-With-Word-Vectors-Trained-With-Model)
0. Complete one of the two [Exercises](#Exercises). Remember to keep notes about what you do!

## Extra Details -- Do Not Do This
This took awhile, which is why I'm giving you the trained file rather than having you do this. But just in case you're curious, here is how to create the trained model file.
1. Download the corpus of sentences from [http://www.statmt.org/lm-benchmark/1-billion-word-language-modeling-benchmark-r13output.tar.gz](http://www.statmt.org/lm-benchmark/1-billion-word-language-modeling-benchmark-r13output.tar.gz)
1. Unzip and unarchive the file: `tar zxf 1-billion-word-language-modeling-benchmark-r13output.tar.gz` 
1. Run the following Python code:
    ```
    from gensim.models import word2vec
    import os

    corpus_dir = '1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled'
    sentences = word2vec.PathLineSentences(corpus_dir)
    model = word2vec.Word2Vec(sentences) # just use all of the default settings for now
    model.save('1billion_word_vectors')
    ```

## Documentation/Sources
* [https://radimrehurek.com/gensim/models/word2vec.html](https://radimrehurek.com/gensim/models/word2vec.html) for more information about how to use gensim word2vec in general
* _Blog post has been removed_ [https://codekansas.github.io/blog/2016/gensim.html](https://codekansas.github.io/blog/2016/gensim.html) for information about using it to create embedding layers for neural networks.
* [https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/) for information on sequence classification with keras
* [https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) for using pre-trained embeddings with keras (though the syntax they use for the model layers is different than most other tutorials).
* [https://keras.io/](https://keras.io/) Keras API documentation

In [305]:
from gensim.models import word2vec
import numpy as np
import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Flatten, ReLU, GlobalMaxPooling1D
from tensorflow.keras import layers

## Load Trained Word Vectors

Load the trained model file into memory

In [306]:
wv_model = word2vec.Word2Vec.load('1billion_word_vectors')

Since we do not need to continue training the model, we can save memory by keeping the parts we need (the word vectors themselves) and getting rid of the rest of the model.

In [307]:
wordvec = wv_model.wv
del wv_model

## Explore Word Vectors
Now we can look at some of the relationships between different words.

Like [the gensim documentation](https://radimrehurek.com/gensim/models/word2vec.html), let's start with a famous example: king + woman - man

In [308]:
wordvec.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.8407386541366577),
 ('monarch', 0.7541723847389221),
 ('prince', 0.7350203394889832),
 ('princess', 0.696908175945282),
 ('empress', 0.6771803498268127),
 ('sultan', 0.6649758815765381),
 ('Chakri', 0.6451102495193481),
 ('goddess', 0.6439394950866699),
 ('ruler', 0.6275452971458435),
 ('kings', 0.6273427605628967)]

This next one does not work as well as I'd hoped, but it gets close. Maybe you can find a better example.

In [309]:
wordvec.most_similar(positive=['panda', 'eucalyptus'], negative=['bamboo'])

[('okapi', 0.7140713334083557),
 ('gibbon', 0.7034620046615601),
 ('koala', 0.697202742099762),
 ('cub', 0.690765917301178),
 ('tortoise', 0.6886162757873535),
 ('beetle', 0.6859476566314697),
 ('salamander', 0.6855185627937317),
 ('psyllid', 0.6837549805641174),
 ('lynx', 0.6802859902381897),
 ('carnivore', 0.6794543266296387)]

Which one of these is not like the others?

Note: It looks like the gensim code needs to be updated to meet the requirements of later versions of numpy. You can ignore the warning.

In [310]:
wordvec.doesnt_match(['red', 'purple', 'laptop', 'turquoise', 'ruby'])

'laptop'

How far apart are different words?

In [311]:
wordvec.distances('laptop', ['computer', 'phone', 'rabbit'])

array([0.205414  , 0.36557418, 0.6597437 ], dtype=float32)

Let's see what one of these vectors actually looks like.

In [312]:
wordvec['textbook']

array([ 0.50756323, -2.8890731 ,  0.9743826 , -0.60089743, -0.23762947,
       -2.324566  , -0.64634913, -0.66476715, -2.3432739 ,  1.4446437 ,
       -0.15542823,  1.8248576 ,  1.1309539 , -0.21071543, -0.82512087,
       -0.2773584 , -0.1973424 , -0.5337731 ,  2.1143918 ,  1.0673765 ,
       -0.2341243 ,  1.5292411 ,  0.66977274,  1.1214821 , -0.57710004,
       -0.02504024,  0.6074397 ,  0.19416903, -1.1265849 , -0.6618393 ,
        1.7525213 ,  1.6232891 , -0.3886833 , -1.1867149 ,  0.45511633,
        1.4240934 , -0.87929034, -1.8920534 ,  2.6986032 , -0.5277589 ,
        2.1202435 ,  0.62670445,  1.0352231 ,  1.4998924 ,  2.5809426 ,
        0.74698585, -0.07757699, -0.67074645,  1.6887746 , -0.22081567,
        1.2107906 ,  0.16741815,  3.3496742 ,  1.1832954 ,  0.4423463 ,
        0.04771314, -0.14557275, -1.3345221 ,  1.3236852 ,  2.0154989 ,
       -0.6510446 ,  0.21808812, -0.31578887, -1.822629  ,  0.8436349 ,
       -1.1500564 ,  1.24044   , -2.6430037 ,  1.0617311 ,  1.20

What other methods are available to us?

In [313]:
help(wordvec)

Help on KeyedVectors in module gensim.models.keyedvectors object:

class KeyedVectors(gensim.utils.SaveLoad)
 |  KeyedVectors(vector_size, count=0, dtype=<class 'numpy.float32'>, mapfile_path=None)
 |  
 |  Serialize/deserialize objects from disk, by equipping them with the `save()` / `load()` methods.
 |  
 |  Warnings
 |  --------
 |  This uses pickle internally (among other techniques), so objects must not contain unpicklable attributes
 |  such as lambda functions etc.
 |  
 |  Method resolution order:
 |      KeyedVectors
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key)
 |  
 |  __getitem__(self, key_or_keys)
 |      Get vector representation of `key_or_keys`.
 |      
 |      Parameters
 |      ----------
 |      key_or_keys : {str, list of str, int, list of int}
 |          Requested key or list-of-keys.
 |      
 |      Returns
 |      -------
 |      numpy.ndarray
 |          Vector representation for `key_or

# Exercise: Explore Word Vectors

## Optional
What other interesting relationship can you find, using the methods used in the examples above or anything you find in the help message?

In [314]:
wordvec.most_similar('computer', topn=10)

[('laptop', 0.7945859432220459),
 ('computers', 0.7850198149681091),
 ('handheld', 0.7827749848365784),
 ('hand-held', 0.7591875195503235),
 ('G.P.S.', 0.7583111524581909),
 ('desktop', 0.75662761926651),
 ('printer', 0.7346934676170349),
 ('software', 0.733142614364624),
 ('server', 0.7329431176185608),
 ('GPS', 0.730388879776001)]

In [315]:
wordvec.closer_than('computer', 'GPS')

['software',
 'computers',
 'laptop',
 'server',
 'desktop',
 'handheld',
 'printer',
 'hand-held',
 'G.P.S.']

## Use Word Vectors in an Embedding Layer of a Keras Model

In [316]:
# get_keras_embedding() was deprecated, instead I use this
# from https://github.com/RaRe-Technologies/gensim/wiki/Using-Gensim-Embeddings-with-Keras-and-Tensorflow

def gensim_to_keras_embedding(keyed_vectors, train_embeddings=False):
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

You may have noticed in the help text for wordvec that it has a built-in method for converting into a Keras embedding layer.

Since for this experimentation, we'll just be giving the embedding layer one word at a time, we can set the input length to 1.

In [317]:
# test_embedding_layer = wordvec.get_keras_embedding()
test_embedding_layer = gensim_to_keras_embedding(wordvec)
test_embedding_layer.input_length = 1

In [318]:
embedding_model = Sequential()
embedding_model.add(test_embedding_layer)

But how do we actually use this? If you look at the [Keras Embedding Layer documentation](https://keras.io/layers/embeddings/) you might notice that it takes numerical input, not strings. How do we know which number corresponds to a particular word? In addition to having a vector, each word has an index:

In [319]:
wordvec.key_to_index['python']

30438

Let's see if we get the same vector from the embedding layer as we get from our word vector object.

In [320]:
wordvec['python']

array([-1.1750487e+00,  2.3066440e-04, -6.0706180e-01, -1.1156354e+00,
       -1.0580894e+00, -2.7154784e+00, -3.6140988e+00, -1.0810910e+00,
        1.1234255e+00, -7.7326834e-01, -1.3322397e+00,  9.2905626e-02,
       -2.4488842e+00, -1.7817341e-01, -3.5459950e+00, -1.7320968e+00,
        1.9397168e+00, -6.3734710e-01,  2.3254216e+00, -1.3535864e+00,
       -1.4451812e-01, -2.4297442e+00,  1.5498929e+00,  8.1969726e-01,
        9.0982294e-01, -6.6116208e-01,  3.8905215e-01,  3.3855909e-01,
       -7.5454485e-01, -1.0352553e+00, -2.5936973e+00,  1.2103225e+00,
       -3.0236175e+00,  3.0580134e+00, -3.9140179e+00,  4.0223894e-01,
        1.7356061e+00,  9.0976155e-01,  2.0956397e-02,  2.0190549e+00,
        4.5332021e-01, -1.6634842e+00, -4.8180079e-01,  2.0414692e-01,
       -5.9267312e-01, -1.4182589e+00, -9.7301149e-01,  5.1611459e-01,
        2.0727324e+00,  2.0064230e+00, -7.5027935e-02, -1.1723986e+00,
       -8.6943096e-01,  1.7028141e+00,  2.2190344e+00,  9.3605727e-01,
      

In [321]:
embedding_model.predict(numpy.array([[30438]]))

array([[[-1.1750487e+00,  2.3066440e-04, -6.0706180e-01, -1.1156354e+00,
         -1.0580894e+00, -2.7154784e+00, -3.6140988e+00, -1.0810910e+00,
          1.1234255e+00, -7.7326834e-01, -1.3322397e+00,  9.2905626e-02,
         -2.4488842e+00, -1.7817341e-01, -3.5459950e+00, -1.7320968e+00,
          1.9397168e+00, -6.3734710e-01,  2.3254216e+00, -1.3535864e+00,
         -1.4451812e-01, -2.4297442e+00,  1.5498929e+00,  8.1969726e-01,
          9.0982294e-01, -6.6116208e-01,  3.8905215e-01,  3.3855909e-01,
         -7.5454485e-01, -1.0352553e+00, -2.5936973e+00,  1.2103225e+00,
         -3.0236175e+00,  3.0580134e+00, -3.9140179e+00,  4.0223894e-01,
          1.7356061e+00,  9.0976155e-01,  2.0956397e-02,  2.0190549e+00,
          4.5332021e-01, -1.6634842e+00, -4.8180079e-01,  2.0414692e-01,
         -5.9267312e-01, -1.4182589e+00, -9.7301149e-01,  5.1611459e-01,
          2.0727324e+00,  2.0064230e+00, -7.5027935e-02, -1.1723986e+00,
         -8.6943096e-01,  1.7028141e+00,  2.2190344

Looks good, right? But let's not waste our time when the computer could tell us definitively and quickly:

In [322]:
embedding_model.predict(numpy.array([[wordvec.key_to_index['python']]]))[0][0] == wordvec['python']

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

Now we have a way to turn words into word vectors with Keras layers. Yes! Time to get some data.

## IMDB Dataset
The [IMDB dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) consists of movie reviews that have been marked as positive or negative. (There is also a built-in dataset of [Reuters newswires](https://keras.io/datasets/#reuters-newswire-topics-classification) that have been classified by topic.)

In [323]:
(x_train, y_train), (x_test, y_test) = imdb.load_data()

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/Users/paul/Downloads/ml_class/ml_env/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/paul/Downloads/ml_class/ml_env/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differen

It looks like our labels consist of 0 or 1, which makes sense for positive and negative.

In [324]:
print(y_train[0:9])
print(max(y_train))
print(min(y_train))

[1 0 0 1 0 0 1 0 1]
1
0


But x is a bit more trouble. The words have already been converted to numbers -- numbers that have nothing to do with the word embeddings we spent time learning!

In [325]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

Looking at the help page for imdb, it appears there is a way to get the word back. Phew.

In [326]:
help(imdb)

Help on module keras.datasets.imdb in keras.datasets:

NAME
    keras.datasets.imdb - IMDB sentiment classification dataset.

FILE
    /Users/paul/Downloads/ml_class/ml_env/lib/python3.8/site-packages/keras/datasets/imdb.py




In [327]:
imdb_offset = 3
imdb_map = dict((index + imdb_offset, word) for (word, index) in imdb.get_word_index().items())
imdb_map[0] = 'PADDING'
imdb_map[1] = 'START'
imdb_map[2] = 'UNKNOWN'
imdb_map

{34704: 'fawn',
 52009: 'tsukino',
 52010: 'nunnery',
 16819: 'sonja',
 63954: 'vani',
 1411: 'woods',
 16118: 'spiders',
 2348: 'hanging',
 2292: 'woody',
 52011: 'trawling',
 52012: "hold's",
 11310: 'comically',
 40833: 'localized',
 30571: 'disobeying',
 52013: "'royale",
 40834: "harpo's",
 52014: 'canet',
 19316: 'aileen',
 52015: 'acurately',
 52016: "diplomat's",
 25245: 'rickman',
 6749: 'arranged',
 52017: 'rumbustious',
 52018: 'familiarness',
 52019: "spider'",
 68807: 'hahahah',
 52020: "wood'",
 40836: 'transvestism',
 34705: "hangin'",
 2341: 'bringing',
 40837: 'seamier',
 34706: 'wooded',
 52021: 'bravora',
 16820: 'grueling',
 1639: 'wooden',
 16821: 'wednesday',
 52022: "'prix",
 34707: 'altagracia',
 52023: 'circuitry',
 11588: 'crotch',
 57769: 'busybody',
 52024: "tart'n'tangy",
 14132: 'burgade',
 52026: 'thrace',
 11041: "tom's",
 52028: 'snuggles',
 29117: 'francesco',
 52030: 'complainers',
 52128: 'templarios',
 40838: '272',
 52031: '273',
 52133: 'zaniacs',

The knowledge about the initial indices and offset came from [this stack overflow post](https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset) after I got gibberish when I tried to translate the first review, below. It looks coherent now!

In [328]:
' '.join([imdb_map[word_index] for word_index in x_train[0]])

"START this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shou

## Train IMDB Word Vectors
The word vectors from the 1 billion words dataset might work for us when trying to classify the IMDB data. Word vectors trained on the IMDB data itself might work better, though.

In [329]:
train_sentences = [['PADDING'] + [imdb_map[word_index] for word_index in review] for review in x_train]
test_sentences = [['PADDING'] + [imdb_map[word_index] for word_index in review] for review in x_test]

In [330]:
# min count says to put any word that appears at least once into the vocabulary
# size sets the dimension of the output vectors
imdb_wv_model = word2vec.Word2Vec(train_sentences + test_sentences + ['UNKNOWN'], min_count=1, vector_size=100)

In [331]:
imdb_wordvec = imdb_wv_model.wv
del imdb_wv_model

## Process Dataset
For this exercise, we're going to keep all inputs the same length (we'll see how to do variable-length later). This means we need to choose a maximum length for the review, cutting off longer ones and adding padding to shorter ones. What should we make the length? Let's understand our data.

In [332]:
lengths = [len(review) for review in x_train + x_test]
print('Longest review: {} Shortest review: {}'.format(max(lengths), min(lengths)))

Longest review: 2697 Shortest review: 70


2697 words! Wow. Well, let's see how many reviews would get cut off at a particular cutoff.

In [333]:
cutoff = 500
print('{} reviews out of {} are over {}.'.format(
    sum([1 for length in lengths if length > cutoff]), 
    len(lengths), 
    cutoff))

8485 reviews out of 25000 are over 500.


In [334]:
x_train_padded = sequence.pad_sequences(x_train, maxlen=cutoff)
x_test_padded = sequence.pad_sequences(x_test, maxlen=cutoff)

## Classification With Word Vectors Trained With Model
Model definition. The embedding layer here learns the 100-dimensional vector embedding within the overall classification problem training. That is usually what we want, unless we have a bunch of un-tagged data that could be used to train word vectors but not a classification model.

In [335]:
not_pretrained_model = Sequential()
not_pretrained_model.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
not_pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
not_pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
not_pretrained_model.add(Flatten())
not_pretrained_model.add(Dense(units=128, activation='relu'))
not_pretrained_model.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
not_pretrained_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

Train the model. __This takes awhile. You might not want to re-run it.__

In [336]:
not_pretrained_model.fit(x_train_padded, y_train, epochs=1, batch_size=64)

391/391 [==============================] - 29s 73ms/step - loss: 0.5108 - binary_accuracy: 0.6991


Assess the model. __This takes awhile. You might not want to re-run it.__

In [337]:
not_pretrained_scores = not_pretrained_model.evaluate(x_test_padded, y_test)
print('loss: {} accuracy: {}'.format(*not_pretrained_scores))

782/782 [==============================] - 9s 11ms/step - loss: 0.2603 - binary_accuracy: 0.8932
loss: 0.2602842450141907 accuracy: 0.8932399749755859


# Exercises

These exercises will help you learn more about how to use word vectors in a model and how to translate between data representations.

For any model that you try in these exercises, take notes about the performance you see and anything you notice about the differences between the models.

## Exercise Option #1 - Advanced Difficulty
Using the details above about how the imdb dataset and the keras embedding layer represent words, define a model that uses the pre-trained word vectors from the imdb dataset rather than an embedding that keras learns as it goes along. You'll need to replace the embedding layer and feed in different training data.

## Classification With Pre-Trained Word Vectors

In [338]:
# create a simple dict containing each word and an index

word_index = {}
word_index = dict(zip(imdb_map.values(), range(len(imdb_map))))
word_index

{'fawn': 0,
 'tsukino': 1,
 'nunnery': 2,
 'sonja': 3,
 'vani': 4,
 'woods': 5,
 'spiders': 6,
 'hanging': 7,
 'woody': 8,
 'trawling': 9,
 "hold's": 10,
 'comically': 11,
 'localized': 12,
 'disobeying': 13,
 "'royale": 14,
 "harpo's": 15,
 'canet': 16,
 'aileen': 17,
 'acurately': 18,
 "diplomat's": 19,
 'rickman': 20,
 'arranged': 21,
 'rumbustious': 22,
 'familiarness': 23,
 "spider'": 24,
 'hahahah': 25,
 "wood'": 26,
 'transvestism': 27,
 "hangin'": 28,
 'bringing': 29,
 'seamier': 30,
 'wooded': 31,
 'bravora': 32,
 'grueling': 33,
 'wooden': 34,
 'wednesday': 35,
 "'prix": 36,
 'altagracia': 37,
 'circuitry': 38,
 'crotch': 39,
 'busybody': 40,
 "tart'n'tangy": 41,
 'burgade': 42,
 'thrace': 43,
 "tom's": 44,
 'snuggles': 45,
 'francesco': 46,
 'complainers': 47,
 'templarios': 48,
 '272': 49,
 '273': 50,
 'zaniacs': 51,
 '275': 52,
 'consenting': 53,
 'snuggled': 54,
 'inanimate': 55,
 'uality': 56,
 'bronte': 57,
 'errors': 58,
 'dialogs': 59,
 "yomada's": 60,
 "madman's": 61

In [339]:
# make a matrix with the pre-trained vectors for each index
# to be input to the Embedding layer

num_tokens = len(word_index) + 2
embedding_dim = 100
hits = 0
misses = 0

imdb_map_inv = {v: k for k, v in imdb_map.items()}

# Prepare embedding matrix

embedding_matrix = np.zeros((num_tokens, embedding_dim))

for word, i in word_index.items():
    embedding_vector = imdb_map_inv.get(word)
    embedding_matrix[i] = embedding_vector
    
print("Converted %d words (%d misses)" % (hits, misses))

Converted 0 words (0 misses)


In [340]:
# create the Embedding layer with the pre-trained vectors
# trainable=False to keep the embeddings fixed

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    input_length=cutoff
)

In [341]:
# define the model

pretrained_model = Sequential()
pretrained_model.add(embedding_layer)
pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
pretrained_model.add(Flatten())
pretrained_model.add(Dense(units=128, activation='relu', ))
pretrained_model.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
pretrained_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [342]:
# fit the model

pretrained_model.fit(x_train_padded, y_train, epochs=1, batch_size=64)

391/391 [==============================] - 16s 41ms/step - loss: 19395.7466 - binary_accuracy: 0.4967


In [343]:
# assess the model

pretrained_scores = pretrained_model.evaluate(x_test_padded, y_test)
print('loss: {} accuracy: {}'.format(*pretrained_scores))

782/782 [==============================] - 10s 12ms/step - loss: 6.5471 - binary_accuracy: 0.4907
loss: 6.547122001647949 accuracy: 0.4906800091266632


## Analysis
I used [this Keras article](https://keras.io/examples/nlp/pretrained_word_embeddings/) to create the pre-trained embedding layer. I used the same model "structure" as the non-pre-trained model, because I wanted to be able to compare the results better—the results being that the pre-trained model scores 50% accuracy, the worst possible score.





## Exercise Option #2 - Advanced Difficulty
Same as option 1, but try using the 1billion vector word embeddings instead of the imdb vectors. If you also did option 1, comment on how the performance changes.